# Размещение баннеров

In [3]:
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift
# from geopy.distance import great_circle

## Load data

In [5]:
df = pd.read_csv("checkins.dat", sep='|', na_values = ' ', skiprows = 2, 
                 names = ['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at'])

C:\Users\Nikolay\AppData\Local\Continuum\Anaconda3\envs\py27\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916.0,5222.0,,,2012-04-21 17:39:01
1,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
2,984315,1764391.0,5222.0,,,2012-04-21 17:37:18
3,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
4,984249,2146840.0,5222.0,,,2012-04-21 17:42:58


## Transforming data

In [7]:
df.latitude = [str(x).replace(' ', '') for x in df.latitude]
df.longitude = [str(x).replace(' ', '') for x in df.longitude]
df = df[df.latitude != '']
df = df[df.latitude != 'nan']
df = df[df.longitude != 'nan']
df = df[df.longitude != '']
df['latitude'] = df['latitude'].astype('float64')
df['longitude'] = df['longitude'].astype('float64')

In [8]:
len(df)

396634

In [9]:
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
7,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
9,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


## Clustering

### Subset first 100k rows

In [10]:
x = df[['latitude', 'longitude']][:100000]

### Train MeanShift

In [11]:
ms = MeanShift(bandwidth=0.1)
ms.fit(x)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

### Get labels and clusters' centers

In [12]:
labels = ms.labels_
cluster_centers = ms.cluster_centers_
x['labels'] = labels
x.head()

,latitude,longitude,labels
1,38.895112,-77.036366,5
3,33.800745,-84.410520,7
7,45.523452,-122.676207,30
9,40.764462,-111.904565,66
10,33.448377,-112.074037,1


### Counting unique clusters

In [13]:
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)
n_clusters_

3229

### Subset clusters with more than 15 items

In [15]:
idx = x['labels'].value_counts()
good_labels = idx.index[idx > 15]
cluster_centers = cluster_centers[good_labels]

### Add offices locations

In [16]:
office = np.array([[33.751277, -118.188740],[25.867736, -80.324116],[51.503016, -0.075479], [52.378894, 4.885084],
                   [39.366487, 117.036146], [-33.868457, 151.205134]])

### Calculate distance

In [217]:
store_dist = []
for i in cluster_centers:
    for j in office:
        store_dist.append([great_circle(i, j).miles, i])

In [18]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

store_dist = []
for cl in cluster_centers:
#     mdist = []
    for co in office:
        store_dist.append([haversine(cl[1], cl[0], co[1], co[0]), cl])

### Sort by miles from offices

In [19]:
store_dist.sort()

### Subset twenty closest locations and sort them

In [248]:
##ORIGIN ANSWER
answer = [x[1] for x in store_dist[:20]]
answer.sort(key=lambda x: x[0])
answer

[array([ -33.86063043,  151.20477593]),
 array([ 25.70534972, -80.28342874]),
 array([ 25.78581242, -80.21793804]),
 array([ 25.84567226, -80.3188906 ]),
 array([ 26.01009825, -80.19999059]),
 array([ 26.12086266, -80.15890668]),
 array([ 26.13884379, -80.33434684]),
 array([ 26.20058464, -80.25071613]),
 array([  33.65089599, -117.75207397]),
 array([  33.67430266, -117.85878927]),
 array([  33.80987796, -118.14892381]),
 array([  33.81730643, -117.89124917]),
 array([  33.87298601, -118.36209115]),
 array([  33.88832534, -118.04892817]),
 array([  33.97257482, -118.16837067]),
 array([  33.98393587, -118.00740497]),
 array([  34.03548695, -118.43899772]),
 array([  34.06039755, -118.24870903]),
 array([ 51.50299126,  -0.12553729]),
 array([ 52.37296399,   4.89231722])]

In [20]:
##MY ANSWER:
answer = [x[1] for x in store_dist[:20]]
answer.sort(key=lambda x: x[0])
answer

[array([ -33.86063043,  151.20477593]),
 array([ 25.70534972, -80.28342874]),
 array([ 25.78581242, -80.21793804]),
 array([ 25.84567226, -80.3188906 ]),
 array([ 26.01009825, -80.19999059]),
 array([ 26.12086266, -80.15890668]),
 array([ 26.13884379, -80.33434684]),
 array([ 26.20058464, -80.25071613]),
 array([  33.65089599, -117.75207397]),
 array([  33.67430266, -117.85878927]),
 array([  33.80987796, -118.14892381]),
 array([  33.81730643, -117.89124917]),
 array([  33.87298601, -118.36209115]),
 array([  33.88832534, -118.04892817]),
 array([  33.97257482, -118.16837067]),
 array([  33.98393587, -118.00740497]),
 array([  34.03548695, -118.43899772]),
 array([  34.06039755, -118.24870903]),
 array([ 51.50299126,  -0.12553729]),
 array([ 52.37296399,   4.89231722])]

### Write answer

In [21]:
final = ''
for i in range(0,20):
    for j in range(0,2):
        final += ' ' + str(answer[i][j])

In [22]:
text_file = open("answer_comb.txt", "w")
text_file.write(final.strip())
text_file.close()

In [23]:
final.strip()

'-33.8606304286 151.204775929 25.7053497211 -80.2834287382 25.78581242 -80.2179380368 25.8456722643 -80.3188905964 26.0100982493 -80.1999905857 26.1208626586 -80.1589066802 26.1388437868 -80.3343468368 26.200584641 -80.2507161256 33.6508959929 -117.752073973 33.6743026598 -117.858789268 33.8098779553 -118.148923807 33.8173064339 -117.891249171 33.8729860116 -118.362091147 33.8883253428 -118.048928172 33.9725748214 -118.168370667 33.983935874 -118.007404973 34.0354869531 -118.438997719 34.0603975546 -118.248709027 51.5029912609 -0.12553728871 52.3729639903 4.89231722258'

In [26]:
def writeans(list_pc1):
    with open("answer_comb.txt", "w") as fout:
        fout.write(" ".join([str(num) for num in list_pc1]))        
writeans(np.array(answer).flatten())

In [27]:
###ANOTHER WAY
def mindist(clcenter, company):
    dist = []
    for co in company:
        dist.append(np.linalg.norm(co - clcenter))
    return np.min(dist)

distances = []
for cl in cluster_centers:
    distances.append(mindist(cl, office))
# mindist(model.cluster_centers_[0], company)
distances = np.array(distances)
distances[:10]

array([ 16.14371999,   6.19395917,   6.29424146,   5.77204798,
        17.59857982,  13.42413138,   6.37826249,   8.885426  ,
        18.91220169,  14.46122737])

In [29]:
ind = np.argsort(distances)[:20]
ind

array([400, 366, 390,  57,  52,  30, 146,  86, 120,  39, 312, 303, 159,
        58,  29,  11,  34, 279,  25,  53], dtype=int64)

In [31]:
coord = cluster_centers[ind]
coord = np.round(coord, decimals=2)
coord

array([[ -3.38600000e+01,   1.51200000e+02],
       [  5.23700000e+01,   4.89000000e+00],
       [  2.58500000e+01,  -8.03200000e+01],
       [  5.15000000e+01,  -1.30000000e-01],
       [  3.38100000e+01,  -1.18150000e+02],
       [  2.57900000e+01,  -8.02200000e+01],
       [  2.57100000e+01,  -8.02800000e+01],
       [  2.60100000e+01,  -8.02000000e+01],
       [  3.38900000e+01,  -1.18050000e+02],
       [  3.38700000e+01,  -1.18360000e+02],
       [  3.39700000e+01,  -1.18170000e+02],
       [  2.61400000e+01,  -8.03300000e+01],
       [  3.39800000e+01,  -1.18010000e+02],
       [  2.61200000e+01,  -8.01600000e+01],
       [  3.38200000e+01,  -1.17890000e+02],
       [  3.40600000e+01,  -1.18250000e+02],
       [  3.36700000e+01,  -1.17860000e+02],
       [  2.62000000e+01,  -8.02500000e+01],
       [  3.40400000e+01,  -1.18440000e+02],
       [  3.41300000e+01,  -1.18120000e+02]])

In [32]:
ind = np.lexsort((coord[:,1],coord[:,0])) 
coord = coord[ind]
coord

array([[ -3.38600000e+01,   1.51200000e+02],
       [  2.57100000e+01,  -8.02800000e+01],
       [  2.57900000e+01,  -8.02200000e+01],
       [  2.58500000e+01,  -8.03200000e+01],
       [  2.60100000e+01,  -8.02000000e+01],
       [  2.61200000e+01,  -8.01600000e+01],
       [  2.61400000e+01,  -8.03300000e+01],
       [  2.62000000e+01,  -8.02500000e+01],
       [  3.36700000e+01,  -1.17860000e+02],
       [  3.38100000e+01,  -1.18150000e+02],
       [  3.38200000e+01,  -1.17890000e+02],
       [  3.38700000e+01,  -1.18360000e+02],
       [  3.38900000e+01,  -1.18050000e+02],
       [  3.39700000e+01,  -1.18170000e+02],
       [  3.39800000e+01,  -1.18010000e+02],
       [  3.40400000e+01,  -1.18440000e+02],
       [  3.40600000e+01,  -1.18250000e+02],
       [  3.41300000e+01,  -1.18120000e+02],
       [  5.15000000e+01,  -1.30000000e-01],
       [  5.23700000e+01,   4.89000000e+00]])

In [33]:
writeans(coord.flatten())